# Homework nr. 2 - data cleaning (deadline 8/11/2018)

In short, the main task is to clean The Metropolitan Museum of Art Open Access dataset.
  
> The instructions are not given in details: It is up to you to come up with ideas on how to fulfill the particular tasks as best you can. ;)

## What are you supposed to do:

  1. Download the dataset MetObjects.csv from the repository https://github.com/metmuseum/openaccess/.
  2. Check consistency of at least three features where you expect problems (include "Object Name" feature).
  3. Select some features where you expect integrity problems (describe your choice) and check integrity of those features.
  4. Convert at least five features to a proper data type. Choose at least one numeric, one categorical and one datetime.
  5. Find some outliers (describe your selection criteria).
  6. Detect missing data in at least three features, convert them to a proper representation (if they are already not), and impute missing values in at least one feature.

**If you do all this properly, you will obtain 6 points**

To earn **extra two points** you can do some of these:
  * Focus more precisely on cleaning of the "Medium" feature. Such if you like to use it in KNN based algorithms later.
  * Focus on the extraction of physical dimensions of each item (width, depth and height in centimeters) from the "Dimensions" feature.

## Comments

  * Please follow the instructions from https://courses.fit.cvut.cz/MI-PDD/homeworks/index.html.
  * If the reviewing teacher is not satisfied, he can give you another chance to rework your homework and to obtain more points.

In [1]:
import numpy as np
import pandas as pd
import sklearn as skit
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from scipy.stats import chi2_contingency
from IPython.display import display, Markdown, Latex

import seaborn as sns
import re

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('openaccess/MetObjects.csv', low_memory=False)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472127 entries, 0 to 472126
Data columns (total 43 columns):
Object Number              472127 non-null object
Is Highlight               472127 non-null bool
Is Public Domain           472127 non-null bool
Object ID                  472127 non-null int64
Department                 472127 non-null object
Object Name                468155 non-null object
Title                      441060 non-null object
Culture                    192200 non-null object
Period                     73147 non-null object
Dynasty                    23304 non-null object
Reign                      11212 non-null object
Portfolio                  21840 non-null object
Artist Role                281379 non-null object
Artist Prefix              96627 non-null object
Artist Display Name        282598 non-null object
Artist Display Bio         235113 non-null object
Artist Suffix              10742 non-null object
Artist Alpha Sort          282569 non-null object


* All of Object ID are different is good set is as row index.

In [4]:
df.index = df['Object ID']

### Check consistency of "Object Name" attribute

In [5]:
df['NameLen'] = df['Object Name'].str.strip().str.len()
display(df['NameLen'].describe())
display(list(df.loc[df.NameLen.nlargest(100).index]['Object Name']))
display(df.loc[df.NameLen.nsmallest(100).index]['Object Name'])
display(list(df['Object Name'].unique()))

count    468155.000000
mean         10.244402
std           8.405991
min           0.000000
25%           5.000000
50%           7.000000
75%          12.000000
max          80.000000
Name: NameLen, dtype: float64

['Double-barreled flintlock shotgun with exchangeable percussion locks and barrels',
 'Nineteen ornamental fittings from a horse harness and an ornamental saddle plate',
 'Blade and for a hatchet (Nata) and mounting for a dagger (Chiisagatana-goshirae)',
 'Set of Pommel (Fuchigashira), Sword grip ornaments (Menuki), and Cord (Kurikata)',
 'Top two lames from a right thigh defense (Cuisse), possibly from armor for a boy',
 'Manuscript (Du Pré, Galliot (fl.1506 -1561) . [Miscelánea] [Manuscrito] . S.XVI)',
 'Plate 15. Descent into Hell; Christ carrying an episcopal cross, entering tPrint',
 'Pall, sheet, Wah, rather coarse, light weight, weft fringe, torn ends, very worn',
 'String of 16 spherical, 1 barrel and 1 cylindrical beads and 2 hippo head amulet',
 'Cone, round, Padiamunnebnesuttawy, chancellor, Amun, lector, priest Beniuutehhor',
 'Scarab, Menkheperra, nsw bit, royal title, mry imn, beloved of Amun, Thutmose II',
 'Cone, Ptahmose, circular impression, high priest of Amun, vizie

Object ID
49198     \r\n
267878    \r\n
378194    \r\n
479408    \r\n
24857       Ax
25577       Ax
32944       Ax
37759       Ax
38627       Ax
38628       Ax
38629       Ax
38630       Ax
38631       Ax
44826       Ax
50575       Ax
50576       Ax
50577       Ax
50594       Ax
50617       Ax
50618       Ax
50622       Ax
50624       Ax
50625       Ax
50626       Ax
50627       Ax
50629       Ax
50631       Ax
50632       Ax
50633       Ax
50634       Ax
          ... 
248056      As
248060      As
248062      As
254565      As
256803      As
500526      Ūd
500580      Po
500581      Po
500582      Lo
500583      Lo
500586      Lo
500587      Lo
500588      Po
500589      Bo
500590      Po
500606      Gu
500607      Gu
500608      Gu
500996      Ūd
501000      Ūd
501092      Ty
503203      Ūd
503211      So
503339      Em
503345      Ūd
503765      Se
504390      Ūd
504702      Yu
505449      Lo
546958      Ax
Name: Object Name, Length: 100, dtype: object

['Coin',
 'Peso',
 'Centavos',
 'Pesos',
 'Bust',
 'Clock',
 'Vase',
 'Side Chair',
 'Figure',
 'Advertisement',
 'Ale glass',
 'Andiron',
 'Apothecary jar',
 'Statuette',
 'Frame',
 'Architectural elements',
 'Loggia elements',
 'Bedroom',
 'Argand lamp',
 'Lamp shade',
 'Armchair',
 'Banister-back armchair',
 'Drawing',
 'Slat-back armchair',
 'Spindle-back armchair',
 'Teapot',
 'Rocking chair',
 'Reclining armchair',
 'Folding armchair',
 'Side chair',
 'Revolving armchair',
 'Furniture hardware',
 'Astral lamp',
 'Window',
 'Aviary',
 "Baby's bottle",
 'Badge',
 'Baking dish',
 'Balcony',
 'Ball',
 'Wall panel',
 'Baluster',
 'Banjo clock',
 'Bank',
 'Baptismal basin',
 'Baptismal bowl',
 'Baptismal certificate',
 'Bottle',
 'Mug',
 'Keg',
 'Basin',
 'Wash basin',
 'Basin Stand',
 'Basin stand',
 'Basket',
 'Door',
 'Oil cruet',
 'Beaker',
 'Pulque beaker',
 'Tumbler',
 'Bean pot',
 'Folding bedstead',
 'Bedstead',
 'Bell',
 'Bell pull',
 'Bellows',
 'Bench',
 'Sleigh seat',
 'Pai

* "Object Name" have a lot of name with \n or \r\n characters.
* In most of the cases I came across, these characters are instead of spaces.
* Second problem in data what I find is ? or (?). 
* This can be used as a result of some uncertainty about the "Object name", 
I will leave this information in data, but I will make the embarrassment consistent with char (?).
* After repair \r\n there is some empty char will replace with nan.
* There are some two char name which can be bad, but there is not documetation for data. I will not replace it.
* Repair consistency of "Object Name" below

In [6]:
q_mark2 = re.compile(r'[?]+')
end_line1 = re.compile(r'\r\n+')
end_line2 = re.compile(r'\n+')
spaces = re.compile(r'[ ]+')
df['Object Name'] = df['Object Name'].str.replace("\(\?\)", '?').str.replace(q_mark2, '(?)')
df['Object Name'] = df['Object Name'].str.replace(end_line1, '\n').str.replace(end_line2, ' ')
df['Object Name'] = df['Object Name'].str.replace(spaces, ' ').str.strip()
df['Object Name'].replace([''], np.nan, inplace=True)

### Next check attribute for consistency is "Dynasty". Here I indicated inconsistencies in the slashes used.

In [7]:
display(df['Dynasty'].unique())

array([nan, 'Dynasty 8–11', 'Dynasty 12', 'early Dynasty 18', 'Dynasty 1',
       'Dynasty 18, early', 'Dynasty 6', 'Dynasty 5', 'Dynasty 4–5',
       'Dynasty 5–6', 'Dynasty 4–mid 5', 'Dynasty 4, mid–5th Dynasty',
       'Dynasty 4', 'Dynasty 3', 'Dynasty 12–13', 'Dynasty 6, first half',
       'Dynasty 6–8', 'Dynasty 6-8', 'Dynasty 4-6', 'early Dynasty 3',
       'Dynasty 11-12', 'Dynasty 18', 'Dynasty 27', 'Dynasty 12–17',
       'Dynasty 17–Early Dynasty 18', 'Dynasty 17–18', 'mid-Dynasty 18',
       'late Dynasty 12–early 13', 'late Dynasty 12–early  Dynasty 13',
       'Dynasty 12, late - early 13', 'Dynasty 13', 'Dynasty 11',
       'Dynasty 12, late–early 13',
       'Dynasty 12, late-13 up to 1700 B. C.', 'Dynasty 13–17',
       'late Dynasty 12–early Dynasty 13',
       'Dynasty 12, late–13 up to 1700 B. C.', 'Dynasty 12, late–13',
       'Dynasty 9 or later', 'Dynasty 9–10', 'Dynasty 9–11', 'Dynasty 17',
       'Second Intermediate Period', 'Dynasty 12, late - 13 up to 1700'

* Especially 
    * 'Dynasty 12, late - early 13'
    * 'Dynasty 12, late–early 13'
    * 'early dynasty 4–5'
    * 'Dynasty 29'
* Word dynasty will be remove, because is in column name
* Convert ? -> (?)
* Conver string to lower case
* Remove th, st, etc. after numbers
* Repair this consistency problem below

In [8]:
q_mark2 = re.compile(r'[?]+')
end_line1 = re.compile(r'\r\n+')
end_line2 = re.compile(r'\n+')
spaces = re.compile(r'[ ]+')
number = df['Dynasty'].nunique()
th = re.compile(r"(?<=\d)(st|nd|rd|th)\b")

df['Dynasty'] = df['Dynasty'].str.lower().str.replace("dynasty", '').str.replace("dyn", '')
df['Dynasty'] = df['Dynasty'].str.replace("\(\?\)", ' ? ').str.replace(q_mark2, ' (?) ')
df['Dynasty'] = df['Dynasty'].str.replace(end_line1, '\n').str.replace(end_line2, ' ')
df['Dynasty'] = df['Dynasty'].str.replace(spaces, ' ').str.strip()
df['Dynasty'] = df['Dynasty'].str.replace("–", '-').str.replace("- ", '-').str.replace(' -', '-').str.strip()
df['Dynasty'] = df['Dynasty'].str.replace(th, '')

display(Markdown("After repair consistency of values the number of unique decrease from \
                 " + str(number) + " to " +  str(df['Dynasty'].nunique())))

After repair consistency of values the number of unique decrease from                  377 to 296

### Next check attribute for consistency is "Medium".

In [9]:
display(list(df['Medium'].unique()))

['Gold',
 'Silver',
 'Bronze or copper',
 'Nickel',
 'Pressed glass',
 'Mahogany, laminated',
 'Gold, amethysts, spessartites, tourmalines, fresh water pearls, quartzes, rock crystal, and enamel',
 'Oak, tulip poplar',
 'Earthenware, basalt',
 'Paper',
 'Brass, iron',
 'Brass',
 'Iron',
 'Bell metal',
 'Bell metal, possibly iron',
 'Iron, brass',
 'Brass, steel',
 'Wrought iron',
 'Brass and iron',
 'Bronze, glass, iron',
 'Earthenware',
 'Earthenware, lusterware',
 'Bronze',
 'Wood, gilt',
 'Wood',
 'Wood, marble',
 'Limestone, ceramic, and Fravrile glass',
 'Sheffield plate, earthenware, glass',
 'Bronze, gilt brass',
 'Gilt bronze',
 'Glass',
 'Oak',
 'Maple, ash',
 'Maple',
 'Hard maple, soft maple, ash',
 'Maple, oak',
 'Graphite on off-white wove paper',
 'Poplar, ash',
 'Maple, cottonwood',
 'Oak, spruce, maple',
 'Pine, ash, maple',
 'Hickory, pine',
 'Beech',
 'Ash',
 'Mahogany',
 'Walnut, white cedar, yellow pine',
 'Maple, birch',
 'Primary: mahogany.  Secondary: cherry (med

* Here is the inconsistency mostly in the font size of the names, the first letter is always big, so for example Iron, oak are different from Oak, iron.
* Another problem is again the characters for line breaks.
* Furthermore, there is two separators "," or "and"
* Repair this consistency problem below

In [10]:
end_line1 = re.compile(r'\r\n+')
end_line2 = re.compile(r'\n+')
spaces = re.compile(r'[ ]+')
df['Medium'] = df['Medium'].str.lower().str.replace(" and ", ',').str.replace(",,", ',')
df['Medium'] = df['Medium'].str.replace(end_line1, '\n').str.replace(end_line2, ' ')
df['Medium'] = df['Medium'].str.replace(', ', ',').str.replace(',', ', ')
df['Medium'] = df['Medium'].str.replace(spaces, ' ').str.strip()

### Features with integrity problems
* I think that data integrity problem can be found for example for attributes "Object Begin Date" and "Object End Date". There end date should be bigger than the start date. These dates are represented by year as int64.

* Next integrity problems could be in Artist attributes. In this attributes are values for more artists separated by "|", so when I have 3 authors separated, there should be  for each of "Role", "Nationality", "Begin Date", "End Data", etc.  exist same count of values separate with "|" too.

* Next may be if the city is in the specific country, region ... But this is necessary check externally.

In [11]:
display((df[(df['Object Begin Date'] > df['Object End Date'])])[['Object Name', 
                                                                 'Object Begin Date', 'Object End Date']].head(10))

number = (df[(df['Object Begin Date'] > df['Object End Date'])])[['Object Name', 
                                                                 'Object Begin Date', 'Object End Date']].shape[0]
display(Markdown("Number of row which has \"Object Begin Date\"\
                 bigger than \"Object End Date\" is " + str(number) + " of " + str(df.shape[0])))
display(Markdown("There are several ways to solve this problem. \
                 For example, by replacing \"Artist End Date\" with value from begin or set to NaN or delete this rows."))

,Object Name,Object Begin Date,Object End Date
Object ID,,,
38468,Yaksha,-45,-55
55128,Print,1861,9
107853,Dolman,1875,1800
209514,Fan,1785,1779
229879,Sample and letter,1785,1773
237874,Fragments,1785,1779
243602,Ring with scarab,0,-5
247380,Lebes gamikos,-399,-3000
249163,Neck-amphora,-450,-475


Number of row which has "Object Begin Date"                 bigger than "Object End Date" is 198 of 472127

There are several ways to solve this problem.                  For example, by replacing "Artist End Date" with value from begin or set to NaN or delete this rows.

In [12]:
x = lambda x: len(str(x).split("|")) if x is not np.nan else 0
number = df[((df['Artist Role'].apply(x) == df['Artist Display Name'].apply(x)) & 
       (df['Artist Display Bio'].apply(x) == df['Artist Alpha Sort'].apply(x)) &
       (df['Artist Begin Date'].apply(x) == df['Artist End Date'].apply(x)))].shape[0]

display(Markdown("Number of row where is OK the count of values in Artist atributes: " 
                 + str(number)))
display(Markdown("The value includes lines where all NaN values are present."))

Number of row where is OK the count of values in Artist atributes: 392990

The value includes lines where all NaN values are present.

In [13]:
#display(list(df['Credit Line'].unique()))
#display(list(df['Dimensions'].unique()))
def split(x):
    x = str(x).split(",")[-1].strip()
    if x.isdigit(): return x
    else: return np.nan
def removeYear(x):
    y = str(x).split(",")[-1].strip()
    if y.isdigit(): return ",".join(str(x).split(",")[:-1])
    else: return x
    
df['Credit Line Year'] = df['Credit Line'].apply(split)
df['Credit Line From'] = df['Credit Line'].apply(removeYear)
df['Credit Line Year'] = pd.to_datetime(df['Credit Line Year'], format='%Y', errors='coerce')

In [16]:
df['Department'] = df['Department'].astype('category')

In [28]:
df['Metadata Date'] = pd.to_datetime(df['Metadata Date'], format='%m/%d/%Y %H:%M:%S %p')

In [29]:
listt = [1,2,3,4]
print(listt[:-1])


[1, 2, 3]


In [ ]:
display(df.info())
display(df.nunique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472127 entries, 1 to 816510
Data columns (total 47 columns):
Object Number              472127 non-null object
Is Highlight               472127 non-null bool
Is Public Domain           472127 non-null bool
Object ID                  472127 non-null int64
Department                 472127 non-null object
Object Name                468151 non-null object
Title                      441060 non-null object
Culture                    192200 non-null object
Period                     73147 non-null object
Dynasty                    23304 non-null object
Reign                      11212 non-null object
Portfolio                  21840 non-null object
Artist Role                281379 non-null object
Artist Prefix              96627 non-null object
Artist Display Name        282598 non-null object
Artist Display Bio         235113 non-null object
Artist Suffix              10742 non-null object
Artist Alpha Sort          282569 non-null object


None